In [2]:
import os
import tensorflow.keras as k
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import ResNet50,ResNet152V2,EfficientNetB4,InceptionResNetV2,VGG16,ResNet101,ResNet101V2,NASNetLarge,Xception, InceptionV3
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.models import load_model
from sklearn.model_selection import StratifiedKFold , KFold ,RepeatedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.preprocessing import image
from tensorflow.keras import layers,models

In [61]:
cd /

/


In [62]:
cd kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset

/kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset


In [63]:
data = pd.read_csv('train.csv')

In [64]:
datagen = ImageDataGenerator(rescale=1/255 ,
        #validation_split=0.05,
        rotation_range=15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') 

In [65]:
train_generator = datagen.flow_from_dataframe(
        dataframe=data,
        directory="./train/",
        x_col="Image",
        y_col="Class",
        #subset="training",
        shuffle=True,
        batch_size=8,
        color_mode="rgb", # Colour images are used.
        class_mode="categorical", # this does the work of encoding
        target_size=(331,331)) # Change acc to model requirments

Found 6469 validated image filenames belonging to 6 classes.


In [66]:
#base_model = EfficientNetB7(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = ResNet101(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = ResNet101V2(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = Xception(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
#base_model = NASNetLarge(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
#base_model = InceptionV3(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
base_model = InceptionResNetV2(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
#base_model = ResNet50(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = ResNet152V2(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
#base_model = VGG16(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')

# This is a try to train not just the last but last 3 layers of the network, 
# this has to be done very carefully at a very low learning rate
for layer in base_model.layers[-4:]:
    layer.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False
model = Sequential([
  base_model, # Pretrained model
 # Last layers that we add manually
  Dense(4096, activation='relu'),
  #Dropout(0.2),
  #Dense(2048, activation='relu'),
  Dropout(0.5),
  Dense(128, activation = 'relu'),
  Dense(6, activation='softmax'),
])

219062272/219055592 [==============================] - 3s 0us/step


In [67]:
optimizer = RMSprop(learning_rate=0.0001, decay = 1e-7)
model.compile(optimizer=optimizer,loss ="categorical_crossentropy", metrics=["accuracy"])

In [68]:
cd /

/


In [69]:
cd kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset

/kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset


In [70]:
model.fit(train_generator,epochs=10, 
                    #validation_data=validation_generator, validation_steps=10,
                      #class_weight=weights,
          steps_per_epoch=6469//8)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
name=[]
y_pred=[]
confidence = []
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
s=0
for i in os.listdir('test/'):
    name.append(i)
    #i='../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'+i
    i = 'test/' + i
    img=image.load_img(i,target_size=(331,331,3))
    img=image.img_to_array(img)/255
    pred=model.predict(img.reshape(1,331,331,3))
    x = np.argmax(pred[0])
    y_pred.append(labels[x])
    confidence.append([pred[0][x],x])
    s+=1
    if s%300==0:
        print(s)
    
output=pd.DataFrame((zip(name,y_pred)),columns=['Image','Class'])
#data.head()

In [ ]:
cd /

In [ ]:
cd kaggle/working/

In [ ]:
output.to_csv('submission.csv',index=False)
output.shape

In [ ]:
import collections
my_list = train_generator.classes
#print("Original List : ",my_list)
ctr = collections.Counter(my_list)
print("Frequency of the elements in the List : ",ctr)

In [ ]:
w0 = 6469/(6*535)
w1 = 6469/(6*593)
w2 = 6469/(6*1539)
w3 = 6469/(6*640)
w4 = 6469/(6*2801)
w5 = 6469/(6*361)

In [ ]:
weights = {0:w0,1:w1,2:w2,3:w3,4:w4,5:w5}

In [ ]:
data.groupby('Class').nunique()

# Method2

In [6]:
df = pd.read_csv('train.csv')

In [7]:
df = df[df.Class != 'Miscellaneous']
df.shape

(3668, 2)

In [8]:
datagen = ImageDataGenerator(rescale=1/255 ,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') 

In [9]:
train_generator = datagen.flow_from_dataframe(
        dataframe=df,
        directory="./train/",
        x_col="Image",
        y_col="Class",
        shuffle=True,
        batch_size=32,
        color_mode="rgb", # Colour images are used.
        class_mode="categorical", # this does the work of encoding
        target_size=(331,331)) # Change acc to model requirments

Found 3668 validated image filenames belonging to 5 classes.


In [19]:
#base_model = EfficientNetB7(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = ResNet101(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = ResNet101V2(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = Xception(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
#base_model = NASNetLarge(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
#base_model = InceptionV3(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
base_model = InceptionResNetV2(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
#base_model = ResNet50(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = ResNet152V2(weights='imagenet',input_shape=(331,331,3),include_top=False,pooling='avg')
#base_model = VGG16(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')

# This is a try to train not just the last but last 3 layers of the network, 
# this has to be done very carefully at a very low learning rate
for layer in base_model.layers[-4:]:
    layer.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False
model = Sequential([
  base_model, # Pretrained model
 # Last layers that we add manually
  Dense(4096, activation='relu'),
 # Dropout(0.2),
#  Dense(2048, activation='relu'),
  Dropout(0.5),
  Dense(128, activation = 'relu'),
  Dense(5, activation='softmax'),
])

In [20]:
optimizer = RMSprop(learning_rate=0.0001, decay = 1e-7)
model.compile(optimizer=optimizer,loss ="categorical_crossentropy", metrics=["accuracy"])

In [12]:
import collections
my_list = train_generator.classes
ctr = collections.Counter(my_list)
print("Frequency of the elements in the List : ",ctr)

Frequency of the elements in the List :  Counter({2: 1539, 3: 640, 1: 593, 0: 535, 4: 361})


In [13]:
w0 = 6469/(6*535)
w1 = 6469/(6*593)
w2 = 6469/(6*1539)
w3 = 6469/(6*640)
w4 = 6469/(6*361)

In [15]:
weights = {0:w0,1:w1,2:w2,3:w3,4:w4}

In [55]:
cd /

/


In [56]:
cd kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset

/kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset


In [21]:
model.fit(train_generator,epochs=10, 
          class_weight=weights,
          steps_per_epoch=3668//32)

Epoch 1/10
114/114 [==============================] - 119s 964ms/step - loss: 1.1956 - accuracy: 0.7313
Epoch 2/10
114/114 [==============================] - 96s 844ms/step - loss: 0.4496 - accuracy: 0.8966
Epoch 3/10
114/114 [==============================] - 96s 835ms/step - loss: 0.3443 - accuracy: 0.9184
Epoch 4/10
114/114 [==============================] - 95s 837ms/step - loss: 0.3826 - accuracy: 0.9186
Epoch 5/10
114/114 [==============================] - 96s 845ms/step - loss: 0.3304 - accuracy: 0.9309
Epoch 6/10
114/114 [==============================] - 96s 840ms/step - loss: 0.3259 - accuracy: 0.9214
Epoch 7/10
114/114 [==============================] - 96s 840ms/step - loss: 0.2961 - accuracy: 0.9321
Epoch 8/10
114/114 [==============================] - 96s 838ms/step - loss: 0.3084 - accuracy: 0.9272
Epoch 9/10
114/114 [==============================] - 96s 837ms/step - loss: 0.2786 - accuracy: 0.9272
Epoch 10/10
114/114 [==============================] - 96s 844ms/step - 

In [57]:
name=[]
y_pred=[]
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
labels[5] = 'Miscellaneous'
s=0
for i in os.listdir('test/'):
    name.append(i)
    #i='../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'+i
    i = 'test/' + i
    img=image.load_img(i,target_size=(331,331,3))
    img=image.img_to_array(img)/255
    pred=model.predict(img.reshape(1,331,331,3))
    k = np.argmax(pred[0])
    if pred[0][k] <0.999:
        k = 5
    y_pred.append(labels[k])
    s+=1
    if s%300==0:
        print(s)
output2=pd.DataFrame((zip(name,y_pred)),columns=['Image','Class'])

300
600
900
1200
1500
1800
2100
2400
2700
3000
3300


In [58]:
cd /

/


In [59]:
cd kaggle/working

/kaggle/working


In [60]:
output2.to_csv('submission2.csv',index=False)
output2.shape

(3489, 2)

In [ ]:
confidence

In [ ]:
new = []
for el in confidence:
    if el[0] < 0.6:
        new.append(el)

In [ ]:
new